# 03 领域术语总混淆？教你构建精准术语词库，提升检索一致性

在RAG系统构建过程中，术语混淆直接影响信息检索的精准度与生成内容的质量。

这主要源于几个方面：
- 向量表示
- 不同行业、公司乃至同一组织内部，都可能存在相似词汇却拥有截然不同含义的情况

这些因素最终导致检索结果偏离预期，大幅降低了答案的质量。

# 一、术语词库构建与维护（Glossary Management）
## 1.1 产生术语混淆的原因

- 术语多义性
- 同义词与近义词
- 领域差异
- 企业专属术语

## 1.2 构建术语词库的目标
术语词库是整个术语一致性优化体系的核心基础设施。

## 1.3 术语词库的构建流程
- Step 1：收集术语来源
- Step 2：标准化术语
- Step 3：建立别名映射关系
- Step 4：添加上下文信息
- Step 5：构建术语索引


一个功能完善的术语词库应包含以下关键字段，以确保其结构化和可操作性：

| 字段名                 | 内容                                                                                   |
|------------------------|----------------------------------------------------------------------------------------|
| 术语（Term）           | 神经网络                                                                              |
| 别名（Synonyms）       | ["人工神经网络", "NN"]                                                                |
| 定义（Definition）     | 神经网络是一种模仿生物神经网络结构和功能的计算模型……                                  |
| 上下文标签（Context Tags） | ["人工智能", "深度学习", "计算机科学"]                                                 |
| 所属领域（Domain）     | 人工智能                                                                              |
| 示例用法（Usage Example） | 在图像识别任务中，我们使用了一个卷积神经网络。                                        |
| 外部链接（External Link） | [维基百科链接](https://en.wikipedia.org/wiki/Artificial_neural_network)               |
| 禁用词/误导词（Stop Words / Misleading Terms） | ["神经系统"（医学中的不同概念）]                                                  |

## 1.4 术语词库与 RAG 集成

- 方式一：预处理阶段替换术语
- 方式二：检索增强
- 方式三：重排序（Re-ranking）
- 方式四：后处理解释

## 1.5 术语词库维护
1. 术语词库结构设计
这是基础，确定词库所需包含的字段和它们之间的关系。

2. 自动抽取术语候选
利用 NLP 工具从大量文本中自动识别和提取潜在的术语。

3. 专家审核与完善
领域专家对自动抽取的术语进行人工审核、修正和补充，确保准确性和专业性。

4. 构建术语关系图谱
如果有需求，可以进一步构建术语之间的层次、关联关系，形成本体（Ontology）或知识图谱（Knowledge Graph），以提升语义理解能力。

5. 版本控制与更新机制建设
建立术语词库的版本管理和定期更新机制，确保其时效性和权威性，应对新术语的出现或旧术语含义的变化。

| 阶段             | 技术名称                                 | 描述                                                                                                                                               |
|------------------|------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| 1. 数据预处理    | 术语抽取、标准化、上下文分块             | 在原始文档和查询进入 RAG 系统之前，识别并提取领域术语，进行统一化处理，并确保文本分块时能有效保留术语的上下文信息。                                   |
| 2. 术语词库构建  | 词库设计、术语关系建模、版本管理         | 建立结构化的术语词库，包含术语、别名、定义、上下文标签等字段。进一步可构建术语间的层级或关联关系（如本体），并建立完善的版本控制与更新机制。           |
| 3. 嵌入与向量化  | 构建术语向量索引、微调领域嵌入模型       | 将术语词库中的标准术语和别名转换为向量，并构建高效的向量索引（如 FAISS）。同时，通过领域适应性训练（如 LoRA）优化通用嵌入模型，使其更好地理解领域特有概念。 |
| 4. 检索增强      | 查询扩展、混合检索、重排序、元数据过滤   | 利用术语词库对用户查询进行扩展（添加别名），结合向量检索与关键词检索（混合检索）。在召回结果后，通过术语匹配度进行重排序，或利用术语作为元数据进行更精确的过滤。 |
| 5. 生成控制      | 提示工程、结构化输出、术语验证           | 设计包含术语词库信息的提示词，引导大模型生成更准确的答案。在输出阶段，可强制模型使用词库中的标准术语，并对生成内容进行术语验证，避免出现混淆或不规范表达。     |
| 6. 评估与反馈    | 术语一致性指标、LLM-as-a-Judge、用户反馈 | 建立专门的评估指标来衡量 RAG 系统在术语一致性方面的表现。利用大型语言模型作为评估器（LLM-as-a-Judge）来检查术语使用情况，并收集用户反馈，持续优化词库和系统。 |

### 玩具版代码

In [29]:
import re
from typing import List, Dict, Any

# 1. 定义术语词库 (保持更新，包含上下文标签)
GLOSSARY = [
    {"term": "卷积神经网络", "synonyms": ["CNN", "卷基神经网络"], "definition": "一种模仿生物神经网络结构和功能的计算模型，特别适用于图像处理。", "context_tags": ["图像识别", "深度学习"]},
    {"term": "机器学习", "synonyms": ["ML", "机器学", "AI算法"], "definition": "一种人工智能领域，使计算机系统能够从数据中学习而无需明确编程。", "context_tags": ["人工智能", "数据科学"]},
    {"term": "自然语言处理", "synonyms": ["NLP", "自然语言"], "definition": "研究人类语言和计算机之间交互的领域。", "context_tags": ["人工智能", "语言学"]},
    {"term": "中央处理器", "synonyms": ["CPU"], "definition": "计算机的算术、逻辑和控制单元。", "context_tags": ["计算机硬件", "电脑"]},
    {"term": "每单位成本", "synonyms": ["CPU"], "definition": "业务分析中衡量每个产品或服务单位的成本。", "context_tags": ["业务分析", "财务管理", "成本"]},
]

class TerminologyProcessor:
    def __init__(self, glossary: List[Dict[str, Any]]):
        self.glossary = glossary
        self.standard_term_map = {}
        self.alias_to_entries_map = {}
        self._build_mappings()

    def _build_mappings(self):
        """构建映射，一个别名可以映射到多个术语条目以处理歧义。"""
        for entry in self.glossary:
            standard_term = entry["term"]
            self.standard_term_map[standard_term.lower()] = standard_term
            
            all_aliases = [standard_term] + entry.get("synonyms", [])
            for alias in all_aliases:
                alias_lower = alias.lower()
                if alias_lower not in self.alias_to_entries_map:
                    self.alias_to_entries_map[alias_lower] = []
                if entry not in self.alias_to_entries_map[alias_lower]:
                    self.alias_to_entries_map[alias_lower].append(entry)

    def standardize_text(self, text: str, context_window: int = 10) -> str:
        """
        使用迭代和替换函数进行上下文感知的术语标准化。
        为不同类型的术语动态生成正确的正则表达式。
        """
        standardized_text = text
        sorted_keys = sorted(self.alias_to_entries_map.keys(), key=len, reverse=True)
        
        for key_lower in sorted_keys:
            possible_entries = self.alias_to_entries_map[key_lower]
            
            # --- 动态为每个key创建正确的正则表达式 ---
            pattern_str = ""
            # 如果key包含英文字母，我们假定它是一个缩写，需要边界判断
            if re.search(r'[a-zA-Z]', key_lower):
                 # 使用环视来确保不匹配单词内部
                 pattern_str = r'(?<![a-zA-Z])' + re.escape(key_lower) + r'(?![a-zA-Z])'
            else:
                 # 对于中文术语，直接精确匹配
                 pattern_str = re.escape(key_lower)

            pattern = re.compile(pattern_str, flags=re.IGNORECASE)

            # 定义一个替换函数，在每次匹配时调用
            def replacer(match: re.Match) -> str:
                if len(possible_entries) == 1:
                    return possible_entries[0]["term"]
                else:
                    # --- 上下文消歧逻辑 ---
                    context_snippet = standardized_text[
                        max(0, match.start() - context_window):
                        min(len(standardized_text), match.end() + context_window)
                    ]
                    for entry in possible_entries:
                        clues = entry.get("context_tags", []) + [entry["term"]]
                        if any(clue in context_snippet for clue in clues):
                            return entry["term"]
                    # 如果没有上下文线索，回退到第一个定义
                    return possible_entries[0]["term"]

            # 使用替换函数更新文本
            standardized_text = pattern.sub(replacer, standardized_text)

        return standardized_text

    def extract_terms(self, text: str) -> List[str]:
        """
        从文本中提取已知的标准术语
        """
        found_terms = set()
        text_lower = text.lower()
        
        for standard_term_lower, original_standard_term in self.standard_term_map.items():
            # 直接进行字符串搜索，不使用 \b
            if re.search(re.escape(standard_term_lower), text_lower):
                found_terms.add(original_standard_term)
                
        return sorted(list(found_terms))


# 1. 使用词库初始化术语处理器。
term_processor = TerminologyProcessor(GLOSSARY)

# 2. 数据预处理阶段：术语标准化
print("--- 2. 数据预处理阶段：术语标准化 ---")
user_query = "我想了解一下ML模型在图像识别中的应用，还有NLP的相关知识。"
processed_query = term_processor.standardize_text(user_query)
print(f"原始查询: {user_query}")
print(f"标准化查询: {processed_query}")

document_text = "最近研究了CNN和AI算法，发现它们在处理大数据方面表现出色，特别是ML在某些场景下的应用。"
processed_document = term_processor.standardize_text(document_text)
print(f"原始文档: {document_text}")
print(f"标准化文档: {processed_document}")

# 3. 术语提取（用于后续的向量化或元数据标记）
print("\n--- 3. 术语提取 ---")
extracted_terms_query = term_processor.extract_terms(processed_query)
print(f"从查询中提取的术语: {extracted_terms_query}")

extracted_terms_document = term_processor.extract_terms(processed_document)
print(f"从文档中提取的术语: {extracted_terms_document}")

# 4. 模拟向量化存储和检索增强（概念性）
print("\n--- 4. 模拟向量化存储和检索增强（概念性） ---")
print("在实际应用中，我们将使用嵌入模型（例如 SentenceTransformers）将标准化后的文本和术语转换为向量。")
print("这些向量随后存储在专门的向量数据库中（如 FAISS、Pinecone 或 Weaviate），以便进行高效的相似性搜索。")
print("在检索过程中，用户查询首先被标准化并向量化，然后用于查询向量数据库以获取相关文档。")

# 5. 模拟检索增强：查询扩展
def enhance_query_for_retrieval(query: str, processor: TerminologyProcessor) -> List[str]:
    """通过术语词库扩展查询关键词以提高召回率。"""
    standardized_query = processor.standardize_text(query)
    query_terms = processor.extract_terms(standardized_query)
    
    expanded_keywords = set([standardized_query])
    for term in query_terms:
        expanded_keywords.add(term)
        for entry in processor.glossary:
            if entry["term"] == term:
                for synonym in entry.get("synonyms", []):
                    expanded_keywords.add(synonym)
                break
    return sorted(list(expanded_keywords))

print("\n--- 5. 模拟检索增强：查询扩展 ---")
original_query_for_retrieval = "我想知道CPU在电脑里的作用，还有成本CPU是什么？"
expanded_keywords = enhance_query_for_retrieval(original_query_for_retrieval, term_processor)
print(f"原始检索查询: {original_query_for_retrieval}")
print(f"扩展后的检索关键词列表: {expanded_keywords}")

print("\n在生产RAG系统中，这些扩展关键词将驱动混合检索策略，结合语义（向量）搜索与基于关键词的搜索，以获得最佳结果。")


--- 2. 数据预处理阶段：术语标准化 ---
原始查询: 我想了解一下ML模型在图像识别中的应用，还有NLP的相关知识。
标准化查询: 我想了解一下机器学习模型在图像识别中的应用，还有自然语言处理的相关知识。
原始文档: 最近研究了CNN和AI算法，发现它们在处理大数据方面表现出色，特别是ML在某些场景下的应用。
标准化文档: 最近研究了卷积神经网络和机器学习习，发现它们在处理大数据方面表现出色，特别是机器学习在某些场景下的应用。

--- 3. 术语提取 ---
从查询中提取的术语: ['机器学习', '自然语言处理']
从文档中提取的术语: ['卷积神经网络', '机器学习']

--- 4. 模拟向量化存储和检索增强（概念性） ---
在实际应用中，我们将使用嵌入模型（例如 SentenceTransformers）将标准化后的文本和术语转换为向量。
这些向量随后存储在专门的向量数据库中（如 FAISS、Pinecone 或 Weaviate），以便进行高效的相似性搜索。
在检索过程中，用户查询首先被标准化并向量化，然后用于查询向量数据库以获取相关文档。

--- 5. 模拟检索增强：查询扩展 ---
原始检索查询: 我想知道CPU在电脑里的作用，还有成本CPU是什么？
扩展后的检索关键词列表: ['CPU', '中央处理器', '我想知道中央处理器在电脑里的作用，还有成本每单位成本是什么？', '每单位成本']

在生产RAG系统中，这些扩展关键词将驱动混合检索策略，结合语义（向量）搜索与基于关键词的搜索，以获得最佳结果。


# 二、数据预处理阶段（Preprocessing）：提升语义表示质量

这是术语一致性优化的“第一道防线”，直接影响后续所有环节的质量。

| 技术名称                          | 描述                                       | 对术语一致性的帮助                           |
|-----------------------------------|--------------------------------------------|----------------------------------------------|
| 术语抽取（NER、TF-IDF、KeyBERT）  | 自动从语料中识别候选术语                   | 提供术语来源，是词库构建的基础               |
| 术语标准化（Term Normalization）  | 替换非标准表达为统一术语（如“AI”→“人工智能”） | 消除输入噪声，确保术语表达一致               |
| 文本清洗与格式统一                | 清理无意义内容、统一大小写、标点等         | 减少干扰，提升术语识别准确率                 |
| 上下文感知分块策略（SemanticChunker） | 按语义相似度切分文本块                     | 保留术语所在上下文信息，避免割裂语义         |

## 2.1 环境准备
首先，确保你安装了必要的Python库：

In [ ]:
! pip install transformers sentence-transformers faiss-cpu scikit-learn spacy
! python -m spacy download zh_core_web_sm
! pip install faiss-cpu

### 步骤一：术语词库结构设计

In [33]:
# 这是一个结构化的JSON，定义了我们知识体系的核心
term_glossary = {
    "神经网络": {
        "synonyms": ["人工神经网络", "NN"],
        "definition": "模仿生物神经网络结构和功能的计算模型",
        "context_tags": ["人工智能", "深度学习"],
        "domain": "计算机科学",
        "stop_words": ["神经系统"]
    },
    "卷积神经网络": {
        "synonyms": ["CNN", "ConvNet"],
        "definition": "通过卷积层提取局部特征的深度学习模型",
        "context_tags": ["计算机视觉", "图像识别"],
        "domain": "人工智能",
        "stop_words": []
    },
    "图像识别": { 
        "synonyms": [],
        "definition": "使计算机识别图像内容的任务",
        "context_tags": ["计算机视觉"],
        "domain": "人工智能",
        "stop_words": []
    },
    "自动驾驶": { 
        "synonyms": [],
        "definition": "使车辆无需人工操作即可自主行驶的技术",
        "context_tags": ["人工智能", "机器人"],
        "domain": "计算机科学",
        "stop_words": []
    },
    "医疗影像诊断": {
        "synonyms": [],
        "definition": "利用医学影像进行疾病诊断",
        "context_tags": ["医疗", "图像处理"],
        "domain": "医学",
        "stop_words": []
    }
}



### 步骤二：2.1 术语抽取与标准化

In [34]:
# 使用spaCy的EntityRuler，根据我们的词库自定义实体识别规则
import spacy

def extract_terms_with_ruler(text, glossary):
    nlp = spacy.load("zh_core_web_sm")
    
    # 创建一个实体规则管道，并从词库中加载所有术语和别名
    ruler = nlp.add_pipe("entity_ruler", before="ner")
    patterns = []
    for term, data in glossary.items():
        patterns.append({"label": "TERM", "pattern": term})
        for syn in data.get("synonyms", []):
            patterns.append({"label": "TERM", "pattern": syn})
    ruler.add_patterns(patterns)
    
    # 处理文本并提取被识别为"TERM"的实体
    doc = nlp(text)
    candidates = {ent.text for ent in doc.ents if ent.label_ == "TERM"}
    return candidates

# 来看一个例子
text = "CNN模型在图像识别中的应用案例包括自动驾驶和医疗影像诊断。"
candidates = extract_terms_with_ruler(text, term_glossary)
print(f"自动抽取的术语候选: {candidates}")

自动抽取的术语候选: {'医疗影像诊断', '图像识别', '自动驾驶', 'CNN'}


# 三、嵌入构建与向量化阶段（Embedding & Vectorization）

核心任务是将这些经过清洗和标准化的术语，转化为机器能够理解和计算的密集向量（Dense Vectors），并构建高效的检索索引。这直接决定了系统语义匹配的能力上限。

| 技术名称                                 | 描述                                       | 对术语一致性的帮助                             |
|------------------------------------------|--------------------------------------------|------------------------------------------------|
| 术语嵌入与向量索引（FAISS / Pinecone）   | 将术语及其别名转换为向量并构建索引         | 支持语义匹配，提升检索时的术语识别能力         |
| 域专用嵌入模型（Legal-BERT、ChatLaw-Text2Vec） | 在专业语料上继续训练通用模型               | 提升术语理解质量，增强语义表示                 |
| Sentence Transformers + PEFT（LoRA）微调 | 参数高效微调嵌入模型                       | 针对特定领域进一步优化术语语义表示             |

### 步骤二：2.2 基于向量相似度的同义词发现

In [47]:
from sentence_transformers import SentenceTransformer, util

# 建议在项目初始化时加载模型，避免重复加载
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2') 

def map_synonyms_by_similarity(main_terms: list, candidates: list, threshold: float = 0.8) -> dict:
    """
    通过计算向量余弦相似度，将候选词映射到最接近的标准术语。

    Args:
        main_terms (list): 标准术语列表。
        candidates (list): 待匹配的候选同义词列表。
        threshold (float): 判断为同义词的相似度阈值。

    Returns:
        dict: 映射成功的同
    """
    _matched_synonyms = {term: [] for term in main_terms}

    if not main_terms or not candidates:
        return _matched_synonyms

    # 批量编码以提升效率
    embeddings = model.encode(main_terms + candidates, convert_to_tensor=True)
    term_embeddings = embeddings[:len(main_terms)]
    candidate_embeddings = embeddings[len(main_terms):]

    # 计算标准术语与所有候选词的余弦相似度矩阵
    similarity_matrix = util.cos_sim(term_embeddings, candidate_embeddings)

    for i, term in enumerate(main_terms):
        for j, candidate in enumerate(candidates):
            if similarity_matrix[i][j] > threshold:
                _matched_synonyms[term].append(candidate)
    
    return _matched_synonyms

# 示例：
main_terms_to_map = ["卷积神经网络", "神经网络"]
all_possible_synonyms = ["CNN", "ConvNet", "人工神经网络", "NN", "神经系统", "深度学习模型"]

optimized_mapped_synonyms = map_synonyms_by_similarity(main_terms_to_map, all_possible_synonyms)
print("\n优化后匹配的别名:", optimized_mapped_synonyms) 


优化后匹配的别名: {'卷积神经网络': ['人工神经网络', '神经系统', '深度学习模型'], '神经网络': ['人工神经网络', '神经系统']}


### 3.2 步骤三：构建术语向量索引

In [1]:
import os

# 定义你的代理地址
proxy_url = 'http://127.0.0.1:4780'

# 为HTTP和HTTPS流量设置代理
# 大多数模型下载等操作都通过HTTPS，所以 'HTTPS_PROXY' 至关重要
os.environ['HTTP_PROXY'] = proxy_url
os.environ['HTTPS_PROXY'] = proxy_url

print("代理环境变量已设置。")
print(f"HTTP_PROXY: {os.getenv('HTTP_PROXY')}")
print(f"HTTPS_PROXY: {os.getenv('HTTPS_PROXY')}")

# --- 在这里开始你的正常代码 ---
# 例如，现在加载模型，它将通过指定的代理进行下载
from sentence_transformers import SentenceTransformer

# 只有在模型未被缓存时，才会通过代理下载
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
print(f"\n正在尝试加载模型 '{model_name}'...")
try:
    model = SentenceTransformer(model_name)
    print("模型加载/下载成功！")
except Exception as e:
    print(f"加载模型时出错: {e}")
    print("请检查您的代理服务是否正在运行，并且地址和端口是否正确。")

代理环境变量已设置。
HTTP_PROXY: http://127.0.0.1:4780
HTTPS_PROXY: http://127.0.0.1:4780


d:\miniconda3\envs\learnRAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



正在尝试加载模型 'paraphrase-multilingual-MiniLM-L12-v2'...
模型加载/下载成功！


In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

def build_term_vector_index(term_glossary: dict, model: SentenceTransformer) -> tuple:
    """
    将术语词库中的所有术语及其别名转换为向量，并构建FAISS索引。

    Args:
        term_glossary (dict): 结构化的术语词库。键为标准术语，值为包含'synonyms'列表的字典。
        model (SentenceTransformer): 已加载的SentenceTransformer模型实例。

    Returns:
        tuple: (faiss.Index, list) 返回构建好的FAISS索引对象和与之对应的术语列表。
    """
    terms_to_index = []
    # 遍历术语映射字典，收集所有标准术语和别名
    # 修正点：将 term_mapping_dict 修改为 term_glossary
    for standard_term, info in term_glossary.items():
        terms_to_index.append(standard_term)
        if "synonyms" in info and isinstance(info["synonyms"], list):
            terms_to_index.extend(info["synonyms"])
    
    unique_terms_to_index = sorted(list(set(terms_to_index)))
    
    print("正在生成术语向量...")
    embeddings = model.encode(unique_terms_to_index, show_progress_bar=True)
    
    embeddings = embeddings.astype('float32')
    dimension = embeddings.shape[1]
    
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    
    print(f"FAISS索引构建完成。包含 {index.ntotal} 个向量，维度为 {dimension}。")
    return index, unique_terms_to_index


# 1. 加载模型
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

# 2. 准备术语数据
term_mapping_example = {
    "卷积神经网络": {"synonyms": ["CNN", "ConvNet"]},
    "Transformer": {"synonyms": ["transformer", "TRANSFORMER"]},
    "图像识别": {"synonyms": ["图像分类", "视觉识别"]}
}

# 3. 使用修正后的函数进行调用
faiss_index, indexed_term_list = build_term_vector_index(term_mapping_example, model)

# 4. 验证结果
print("\n--- 索引构建成功 ---")
print(f"FAISS 索引中的向量数量: {faiss_index.ntotal}")
print(f"被索引的术语列表: {indexed_term_list}")

正在生成术语向量...


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.71it/s]

FAISS索引构建完成。包含 9 个向量，维度为 384。

--- 索引构建成功 ---
FAISS 索引中的向量数量: 9
被索引的术语列表: ['CNN', 'ConvNet', 'TRANSFORMER', 'Transformer', 'transformer', '卷积神经网络', '图像分类', '图像识别', '视觉识别']


如何调用该函数并获取索引和术语列表的示例

In [8]:
# --- 第2部分：定义我们的核心检索函数 ---

def search_similar_terms(query_text: str, model: SentenceTransformer, index: faiss.Index, term_list: list, k: int = 5):
    """
    在FAISS索引中检索与查询文本最相似的k个术语。

    Args:
        query_text (str): 用户输入的查询词。
        model (SentenceTransformer): 用于编码查询词的模型。
        index (faiss.Index): FAISS索引对象。
        term_list (list): 与索引向量顺序一致的术语列表。
        k (int): 希望返回的最相似结果的数量。
    """
    print(f"\n--- 正在执行检索 ---")
    print(f"查询: '{query_text}'")
    
    # 1. 将查询文本编码为向量
    query_vector = model.encode([query_text])
    query_vector = query_vector.astype('float32')
    
    # 2. 在FAISS索引中执行搜索
    # index.search返回两个数组：D (distances) 和 I (indices)
    distances, indices = index.search(query_vector, k)
    
    # 3. 解析并打印结果
    print("检索结果:")
    for i in range(k):
        idx = indices[0][i]
        dist = distances[0][i]
        term = term_list[idx]
        
        # 对于IndexFlatL2，距离是平方欧氏距离，距离越小代表越相似
        print(f"  Top {i+1}: 术语='{term}',  距离={dist:.4f} (值越小越相似)")


# 4. === 演示效果 ===

# **案例一：用别名查询标准术语**
# 目标：测试系统能否理解 "CNN" 就是 "卷积神经网络"。
search_similar_terms(query_text="CNN", model=model, index=faiss_index, term_list=indexed_term_list, k=3)

# **案例二：语义相近查询（核心优势展示）**
# 目标：查询一个不在我们词库中，但意思相近的词 "计算机视觉"。
# 预期：系统应该能找到 "图像识别" 或 "视觉识别" 等相关术语。
search_similar_terms(query_text="计算机视觉", model=model, index=faiss_index, term_list=indexed_term_list, k=3)

# **案例三：用一个更宽泛的词查询**
# 目标：查询 "语言模型"，看是否能找到更具体的 "大型语言模型" 或 "自然语言处理"。
search_similar_terms(query_text="语言模型", model=model, index=faiss_index, term_list=indexed_term_list, k=3)

# **案例四：测试对轻微噪声的容忍度**
# 目标：查询一个不存在的、略有差异的词 "变换器模型"，看是否能正确匹配到 "Transformer模型"。
search_similar_terms(query_text="变换器模型", model=model, index=faiss_index, term_list=indexed_term_list, k=3)



--- 正在执行检索 ---
查询: 'CNN'
检索结果:
  Top 1: 术语='CNN',  距离=0.0000 (值越小越相似)
  Top 2: 术语='图像分类',  距离=35.1526 (值越小越相似)
  Top 3: 术语='ConvNet',  距离=35.1713 (值越小越相似)

--- 正在执行检索 ---
查询: '计算机视觉'
检索结果:
  Top 1: 术语='视觉识别',  距离=11.4270 (值越小越相似)
  Top 2: 术语='图像识别',  距离=14.5564 (值越小越相似)
  Top 3: 术语='图像分类',  距离=15.7114 (值越小越相似)

--- 正在执行检索 ---
查询: '语言模型'
检索结果:
  Top 1: 术语='图像分类',  距离=24.3401 (值越小越相似)
  Top 2: 术语='卷积神经网络',  距离=26.5247 (值越小越相似)
  Top 3: 术语='图像识别',  距离=27.9374 (值越小越相似)

--- 正在执行检索 ---
查询: '变换器模型'
检索结果:
  Top 1: 术语='TRANSFORMER',  距离=11.7908 (值越小越相似)
  Top 2: 术语='图像分类',  距离=17.0967 (值越小越相似)
  Top 3: 术语='卷积神经网络',  距离=17.4776 (值越小越相似)


# 四、检索增强阶段

核心目标是在初步召回（Recall）的基础上，进一步优化检索结果的广度与精度。预处理阶段解决了术语的“标准”问题，而本阶段则聚焦于如何利用这些标准化的知识，在实际检索中发挥最大效用。

| 技术名称                             | 描述                                                                                   | 对术语一致性的帮助                                                                 |
|--------------------------------------|----------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------|
| 查询扩展与重写（MultiQueryRetriever） | 利用 LLM 生成多个语义等价的查询变体，合并检索结果。                                     | 自动覆盖用户未提及的同义词或相关表达，极大提升对多样化术语的识别与召回能力。         |
| HyDE（假设文档嵌入）                 | 利用 LLM 为查询生成一个“理想答案”的假设性文档，再用该文档的嵌入进行检索。               | 通过生成富含上下文的理想答案，有效缓解原始查询中术语模糊或信息不足的问题，提升检索相关性。 |
| 混合检索（BM25 + FAISS）             | 结合关键词检索（如 BM25）与向量检索（如 FAISS）的优势。                                 | 综合利用字面精确匹配和语义相似匹配，确保基础术语不丢失，同时发现语义相关内容。         |
| 交叉编码器重排序（BGE-reranker）     | 使用更复杂的交叉编码器模型（如 BGE-reranker）对召回结果进行精细化重排序。               | 通过深度交互分析查询与文档的匹配度，提升对术语匹配度的排序精度

In [ ]:
! pip install langchain_community
! pip install langchain langchain-openai faiss-cpu sentence-transformers

### 4.1 核心技术一：查询扩展与重写

In [12]:
## 查询扩展与重写
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os

# --- 准备工作：设置API Key并创建向量数据库 ---

# 设置您的OpenAI API Key
# os.environ["OPENAI_API_KEY"] = "sk-..."

# 1. 准备示例文档
# 我们创建一些包含专业术语的示例文本
doc_text = """
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出色。
它的核心在于通过卷积层和池化层自动提取图像的局部特征。

与CNN不同，Transformer模型最初应用于自然语言处理（NLP）任务，
例如机器翻译。如今，它也被成功应用于计算机视觉，称为Vision Transformer。

大型语言模型（LLM）是当前AI研究的热点，它基于Transformer架构，
能够理解和生成类似人类的文本，展现出强大的推理能力。
"""
with open("sample_tech_doc.txt", "w", encoding="utf-8") as f:
    f.write(doc_text)

# 2. 加载和切分文档
loader = TextLoader("sample_tech_doc.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

# 3. 创建向量数据库
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# --- MultiQueryRetriever 实现 ---

# 4. 初始化LLM和检索器
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# 5. 执行查询
query = "CNN是什么？"
retrieved_docs = retriever_from_llm.invoke(query)

# --- 效果分析 ---
print(f"原始查询: {query}")
print("\n--- MultiQueryRetriever 生成的查询变体 ---")
# MultiQueryRetriever 内部有日志记录生成的查询，这里我们手动展示其可能生成的查询
# 实际使用中可以通过设置 logging.basicConfig(level=logging.INFO) 查看
generated_queries = [
    "卷积神经网络的定义是什么？",
    "CNN模型在深度学习中的作用是什么？",
    "介绍一下卷积神经网络（CNN）。"
]
for i, q in enumerate(generated_queries):
    print(f"查询变体 {i+1}: {q}")

print("\n--- 最终检索到的文档内容 ---")
for doc in retrieved_docs:
    print(doc.page_content)

原始查询: CNN是什么？

--- MultiQueryRetriever 生成的查询变体 ---
查询变体 1: 卷积神经网络的定义是什么？
查询变体 2: CNN模型在深度学习中的作用是什么？
查询变体 3: 介绍一下卷积神经网络（CNN）。

--- 最终检索到的文档内容 ---
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出色。
它的核心在于通过卷积层和池化层自动提取图像的局部特征。
与CNN不同，Transformer模型最初应用于自然语言处理（NLP）任务，
例如机器翻译。如今，它也被成功应用于计算机视觉，称为Vision Transformer。
大型语言模型（LLM）是当前AI研究的热点，它基于Transformer架构，
能够理解和生成类似人类的文本，展现出强大的推理能力。


### 4.2 假设性文档嵌入 (HyDE)

In [ ]:
! pip install --upgrade langchain langchain-community langchain-openai rank_bm25 faiss-cpu

In [50]:
import os
from langchain.retrievers import MergerRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# --- 1. 准备工作：设置API Key并准备数据 ---

# os.environ["OPENAI_API_KEY"] = "sk-..."

# 准备示例文档，我们稍微丰富一下内容以便于对比
doc_text = """
第一部分：关于卷积网络。
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出色。
它的核心在于通过卷积层和池化层自动提取图像的局部特征。CNN的这个特性让它非常高效。

第二部分：关于Transformer。
与CNN不同，Transformer模型最初应用于自然语言处理（NLP）任务，
例如机器翻译。如今，一种被称为Vision Transformer（ViT）的变体也被成功应用于计算机视觉领域。

第三部分：关于大模型。
大型语言模型（LLM）是当前AI研究的热点，它通常基于Transformer架构，
能够理解和生成类似人类的文本，展现出强大的推理能力。
"""
with open("hybrid_search_doc.txt", "w", encoding="utf-8") as f:
    f.write(doc_text)

# 加载和切分文档
loader = TextLoader("hybrid_search_doc.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

print(f"文档已被切分为 {len(docs)} 个块。")


# --- 2. 构建两个不同的检索器 ---

# **检索器一：FAISS 向量检索器 (用于语义匹配)**
print("\n正在构建FAISS向量检索器...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("FAISS检索器构建完成。")


# **检索器二：BM25 关键词检索器 (用于精确匹配)**
print("\n正在构建BM25关键词检索器...")
# BM25Retriever可以直接从文档列表初始化，它不需要嵌入模型
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 3
print("BM25检索器构建完成。")


# --- 3. 使用 MergerRetriever 合并 ---

print("\n正在初始化 MergerRetriever...")
# 创建一个检索器列表
retriever_list = [bm25_retriever, faiss_retriever]

# 初始化 MergerRetriever
# 它会自动处理并行检索和结果去重
merged_retriever = MergerRetriever(retrievers=retriever_list)
print("MergerRetriever 初始化完成。")


# --- 4. 执行查询并对比效果 ---

query = "ViT的技术细节"
print(f"\n\n--- 正在执行混合检索 ---")
print(f"查询: '{query}'")


# **为了对比，我们先看看每个检索器单独工作的结果**
print("\n--- 单独检索结果对比 ---")
bm25_results = bm25_retriever.invoke(query)
print(f"【BM25 关键词检索结果】(共 {len(bm25_results)} 条):")
for doc in bm25_results:
    print(f"  - {doc.page_content[:50]}...")

faiss_results = faiss_retriever.invoke(query)
print(f"\n【FAISS 向量检索结果】(共 {len(faiss_results)} 条):")
for doc in faiss_results:
    print(f"  - {doc.page_content[:50]}...")


# **现在看看 MergerRetriever 的混合结果**
print("\n--- MergerRetriever 混合检索结果 ---")
merged_results = merged_retriever.invoke(query)
print(f"【最终混合结果】(共 {len(merged_results)} 条，已去重):")
for doc in merged_results:
    print(f"  - {doc.page_content[:50]}...")

文档已被切分为 3 个块。

正在构建FAISS向量检索器...
FAISS检索器构建完成。

正在构建BM25关键词检索器...
BM25检索器构建完成。

正在初始化 MergerRetriever...
MergerRetriever 初始化完成。


--- 正在执行混合检索 ---
查询: 'ViT的技术细节'

--- 单独检索结果对比 ---
【BM25 关键词检索结果】(共 3 条):
  - 第三部分：关于大模型。
大型语言模型（LLM）是当前AI研究的热点，它通常基于Transformer...
  - 第二部分：关于Transformer。
与CNN不同，Transformer模型最初应用于自然语言处...
  - 第一部分：关于卷积网络。
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出...

【FAISS 向量检索结果】(共 3 条):
  - 第二部分：关于Transformer。
与CNN不同，Transformer模型最初应用于自然语言处...
  - 第一部分：关于卷积网络。
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出...
  - 第三部分：关于大模型。
大型语言模型（LLM）是当前AI研究的热点，它通常基于Transformer...

--- MergerRetriever 混合检索结果 ---
【最终混合结果】(共 6 条，已去重):
  - 第三部分：关于大模型。
大型语言模型（LLM）是当前AI研究的热点，它通常基于Transformer...
  - 第二部分：关于Transformer。
与CNN不同，Transformer模型最初应用于自然语言处...
  - 第二部分：关于Transformer。
与CNN不同，Transformer模型最初应用于自然语言处...
  - 第一部分：关于卷积网络。
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出...
  - 第一部分：关于卷积网络。
卷积神经网络（CNN）是深度学习中的一种关键模型，尤其在图像识别领域表现出...
  - 第三部分：关于大模型。
大型语言模型（LLM）是当前AI研究的热点，它通常基于Transformer...


## 五、生成控制与输出验证阶段

| 技术名称                  | 描述                                                                 | 对术语一致性的贡献与作用                                                                 |
|---------------------------|----------------------------------------------------------------------|------------------------------------------------------------------------------------------|
| 提示工程 (Prompt Engineering)       | 在提示中明确指令，引导 LLM 使用标准术语、保持特定风格。                | 最基础的控制手段，直接影响 LLM 的选词倾向，引导其遵循术语规范。                           |
| 结构化输出 (Structured Output)     | 强制 LLM 返回符合预定义模式（如 Pydantic 或 JSON Schema）的对象。      | 从根本上杜绝术语的随意使用，确保关键信息以标准、可控的格式输出。                         |
| 输出解析与修复 (Output Parsers)    | 使用如 OutputFixingParser 等工具，在 LLM 输出格式错误时自动尝试修复。  | 提升结构化输出的鲁棒性，能自动纠正轻微的术语格式或拼写错误。                               |
| 后处理与内容增强                   | 在答案文本中自动高亮术语、添加定义弹窗或引用链接。                    | 提升最终答案的可读性和专业性，为用户提供即时的术语解释和来源追溯。                        |
| LLM 即评委 (LLM-as-a-Judge)        | 使用另一个 LLM 实例，根据预设标准（如术语一致性）对生成结果进行打分评估。 | 提供一种可扩展的、自动化的输出质量与术语合规性评估方案。                                  |

### 5.1 生成时控制：结构化输出

In [51]:
import os
from typing import List

# 修正点：直接从 pydantic 库导入 BaseModel 和 Field
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# --- 1. 准备工作 ---
# os.environ["OPENAI_API_KEY"] = "sk-..."
llm = ChatOpenAI(temperature=0, model="gpt-4o")

# --- 2. 定义期望的输出结构 (使用 Pydantic V2) ---
class TerminologyInAnswer(BaseModel):
    """一个包含标准答案和其中使用的技术术语的结构化模型。"""
    answer: str = Field(description="对用户问题的详细、准确的回答。")
    standard_terms_used: List[str] = Field(
        description="在回答中明确使用到的、来自官方词库的标准技术术语列表。",
        example=["卷积神经网络", "图像识别"]
    )

# --- 3. 创建一个结构化输出链 ---
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个精通AI技术的专家。请根据用户的问题，以结构化的形式提供答案。"),
    ("human", "请解释一下什么是CNN，以及它的主要应用领域。")
])

# 使用 .with_structured_output() 将LLM与我们的Pydantic V2模型绑定
# 现在它会默认使用最高效的json_schema模式，而不会产生警告
structured_llm_chain = prompt | llm.with_structured_output(TerminologyInAnswer)

# --- 4. 执行链并查看结果 ---
print("--- 正在执行结构化输出链 ---")
structured_response = structured_llm_chain.invoke({})

print("\n--- LLM返回的结构化对象 ---")
print(structured_response)

print("\n--- 对结果的分析 ---")
print(f"回答内容: {structured_response.answer}")
print(f"模型确认使用的标准术语: {structured_response.standard_terms_used}")

if "卷积神经网络" in structured_response.standard_terms_used:
    print("验证通过：答案中正确地使用了标准术语'卷积神经网络'。")


# --- 5. 内容增强：自动添加术语解释 ---
print("\n" + "="*50)
print("--- 开始执行内容增强 ---")

# 假设我们有这样一个简化的术语词库
glossary = {
    "卷积神经网络": "一种特殊的深度学习模型，擅长处理图像数据。",
    "深度学习": "机器学习的一个分支，它基于人工神经网络。",
    "图像识别": "计算机视觉中的一个核心任务，旨在识别和分类图像中的对象。"
}

# 获取LLM生成的答案文本
llm_answer_text = structured_response.answer

def enhance_text_with_definitions(text: str, term_glossary: dict) -> str:
    """
    在文本中查找标准术语，并为其添加Markdown格式的悬浮注释。
    """
    enhanced_text = text
    for term, definition in term_glossary.items():
        # 创建带注释的Markdown/HTML格式
        replacement = f'<abbr title="{definition}">{term}</abbr>'
        # 替换文本中的术语
        enhanced_text = enhanced_text.replace(term, replacement)
    return enhanced_text

# 执行内容增强
final_output = enhance_text_with_definitions(llm_answer_text, glossary)

print("\n--- 内容增强后的最终输出 (在支持HTML的Markdown渲染器中查看) ---")
print(final_output)



--- 正在执行结构化输出链 ---

--- LLM返回的结构化对象 ---
answer='卷积神经网络（Convolutional Neural Network, CNN）是一种深度学习模型，专门用于处理具有网格结构的数据，例如图像。CNN通过使用卷积层来提取输入数据的特征，通常包括卷积层、池化层和全连接层。卷积层通过卷积运算提取局部特征，池化层用于减少特征图的尺寸，从而降低计算复杂度。\n\n主要应用领域包括：\n\n1. **图像识别**：CNN在图像分类任务中表现出色，能够识别和分类图像中的对象。\n2. **目标检测**：用于在图像中定位和识别多个对象。\n3. **图像分割**：将图像划分为不同的区域或对象。\n4. **自然语言处理**：在文本分类和情感分析等任务中也有应用。\n5. **医学影像分析**：用于分析医学图像，如X光片和MRI扫描。\n6. **自动驾驶**：用于识别道路标志、行人和其他车辆。' standard_terms_used=['卷积神经网络', '图像识别', '目标检测', '图像分割', '自然语言处理', '医学影像分析', '自动驾驶']

--- 对结果的分析 ---
回答内容: 卷积神经网络（Convolutional Neural Network, CNN）是一种深度学习模型，专门用于处理具有网格结构的数据，例如图像。CNN通过使用卷积层来提取输入数据的特征，通常包括卷积层、池化层和全连接层。卷积层通过卷积运算提取局部特征，池化层用于减少特征图的尺寸，从而降低计算复杂度。

主要应用领域包括：

1. **图像识别**：CNN在图像分类任务中表现出色，能够识别和分类图像中的对象。
2. **目标检测**：用于在图像中定位和识别多个对象。
3. **图像分割**：将图像划分为不同的区域或对象。
4. **自然语言处理**：在文本分类和情感分析等任务中也有应用。
5. **医学影像分析**：用于分析医学图像，如X光片和MRI扫描。
6. **自动驾驶**：用于识别道路标志、行人和其他车辆。
模型确认使用的标准术语: ['卷积神经网络', '图像识别', '目标检测', '图像分割', '自然语言处理', '医学影像分析', '自动驾驶']
验证通过：答案中正确地使用了标准术语'卷积神经网络'。

--- 开始执行

### 5.2 生成后处理：验证与内容增强

In [52]:
# 假设我们有这样一个简化的术语词库
glossary = {
    "卷积神经网络": "一种特殊的深度学习模型，擅长处理图像数据。",
    "Transformer架构": "一种基于自注意力机制的神经网络结构，在NLP领域取得巨大成功。",
    "大型语言模型": "基于海量数据训练的、参数规模巨大的语言模型。"
}

# 假设这是LLM返回的、已经验证过的答案文本
llm_answer_text = "大型语言模型通常基于Transformer架构，而卷积神经网络则在图像处理方面是主流。"

def enhance_text_with_definitions(text: str, term_glossary: dict) -> str:
    """
    在文本中查找标准术语，并为其添加Markdown格式的悬浮注释。
    在支持HTML的Markdown渲染器中，这通常会显示为鼠标悬停提示。
    """
    enhanced_text = text
    for term, definition in term_glossary.items():
        # 创建带注释的Markdown/HTML格式
        replacement = f'<abbr title="{definition}">{term}</abbr>'
        # 替换文本中的术语
        enhanced_text = enhanced_text.replace(term, replacement)
    return enhanced_text

# 执行内容增强
final_output = enhance_text_with_definitions(llm_answer_text, glossary)

print("\n--- 内容增强后的输出 ---")
print(final_output)


--- 内容增强后的输出 ---
<abbr title="基于海量数据训练的、参数规模巨大的语言模型。">大型语言模型</abbr>通常基于<abbr title="一种基于自注意力机制的神经网络结构，在NLP领域取得巨大成功。">Transformer架构</abbr>，而<abbr title="一种特殊的深度学习模型，擅长处理图像数据。">卷积神经网络</abbr>则在图像处理方面是主流。


## 六、评估与反馈机制

人工抽样评估难以覆盖海量的生成内容，而“LLM即评委”为此提供了一个高效、可扩展的自动化解决方案。其核心是利用一个LLM的理解和推理能力，来评估另一个LLM（或整个RAG系统）的输出质量。

使用LCEL构建评估链

In [53]:
import os
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# --- 1. 准备工作 ---
# os.environ["OPENAI_API_KEY"] = "sk-..."
evaluator_llm = ChatOpenAI(temperature=0, model="gpt-4o")

# --- 2. 定义评估结果的结构化模型 ---
class TerminologyEvaluation(BaseModel):
    """用于评估术语一致性的结构化模型。"""
    consistency_score: int = Field(description="一个从1到5的分数，5代表完全一致，1代表严重不一致。")
    is_consistent: bool = Field(description="布尔值，是否整体上符合术语规范。")
    reasoning: str = Field(description="对评分的详细解释，指出做得好的地方和存在的问题。")
    suggestions_for_improvement: List[str] = Field(description="为改进答案中的术语使用提出的具体建议。")

# --- 3. 构建评估链 ---
# 创建一个专门的评估提示
evaluation_prompt_template = """
你是一个严格的AI技术文档质量评估员，你的核心任务是评估一段回答在术语使用上的一致性和准确性。

**评估标准:**
1.  **准确性**: 是否正确地使用了标准术语？
2.  **规范性**: 是否避免了使用非官方的、易产生歧义的别名？
3.  **全面性**: 是否在必要时使用了最能表达其意的标准术语？

**权威术语词库 (部分):**
- 卷积神经网络 (别名: CNN)
- Transformer模型 (别名: Transformer)
- 大语言模型 (别名: LLM)

**待评估的回答:**
{answer_text}

请根据以上标准和词库，以JSON格式输出你的评估结果。
"""

prompt = ChatPromptTemplate.from_template(evaluation_prompt_template)
parser = JsonOutputParser(pydantic_object=TerminologyEvaluation)

# 使用LCEL构建评估链
evaluation_chain = prompt | evaluator_llm | parser

# --- 4. 执行评估 ---

# 案例一：一个使用规范的回答
good_answer = "大语言模型（LLM）是基于Transformer模型构建的，而卷积神经网络（CNN）则在图像领域应用广泛。"

# 案例二：一个使用了非标准术语的回答
bad_answer = "大模型是基于变换器架构的，而卷基神经网络在图片处理上很强。"

print("--- 正在评估【优秀回答】---")
good_evaluation_result = evaluation_chain.invoke({"answer_text": good_answer})
print(good_evaluation_result)


print("\n--- 正在评估【待改进回答】---")
bad_evaluation_result = evaluation_chain.invoke({"answer_text": bad_answer})
print(bad_evaluation_result)


--- 正在评估【优秀回答】---
{'准确性': {'评价': '高', '说明': "回答中正确使用了'大语言模型'、'Transformer模型'和'卷积神经网络'等标准术语。"}, '规范性': {'评价': '高', '说明': '回答中避免了使用非官方的、易产生歧义的别名，使用的术语均为官方标准术语。'}, '全面性': {'评价': '高', '说明': '回答在必要时使用了最能表达其意的标准术语，清晰地传达了大语言模型与Transformer模型的关系，以及卷积神经网络的应用领域。'}}

--- 正在评估【待改进回答】---
{'准确性': {'评价': '不准确', '问题': "使用了不正确的术语'卷基神经网络'，应为'卷积神经网络'。"}, '规范性': {'评价': '不规范', '问题': "使用了非官方的术语'大模型'，应为'大语言模型'或其别名'LLM'。"}, '全面性': {'评价': '不全面', '问题': "未使用最能表达其意的标准术语'Transformer模型'，而是使用了非标准的'变换器架构'。"}}


## 总结：术语一致性优化路线图

经过以上各阶段的详细探讨，从数据预处理到最终的评估反馈，我们已经全面构建了保障术语一致性的技术体系。

为了更直观地理解各项技术的定位与优先级，我们将整个优化策略总结为以下分级路线图，为不同阶段的RAG系统建设提供实践指引。

| 优化层级                | 核心技术与解决方案                                                                 |
|-------------------------|------------------------------------------------------------------------------------|
| 基础核心 (Foundation)    | 术语词库构建、术语抽取、预处理标准化、术语嵌入与向量索引                             |
| 关键增强 (Key Enhancement)| 混合检索 (BM25 + 向量)、查询扩展 (MultiQuery)、假设性文档嵌入 (HyDE)、交叉编码器重排序 |
| 辅助优化 (Auxiliary Optimization) | 领域专用嵌入模型微调、上下文感知分块                                               |
| 生成控制 (Generation Control)     | 提示工程、结构化输出、输出解析与修复                                               |
| 长期保障 (Long-term Assurance)    | LLM即评委 (LLM-as-a-Judge)、用户反馈闭环、日志审计与分析                            |